In [1]:
# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import csv
import numpy as np
import matplotlib.pyplot as plt
# display plots in this notebook
%matplotlib inline

# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

In [2]:
# The caffe module needs to be on the Python path;
#  we'll add it here explicitly.
import sys
caffe_root = '/home/ambika/caffe/'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, caffe_root + 'python')

import caffe
# If you get "No module named _caffe", either you have not built pycaffe or you have the wrong path.

In [3]:
import os
if os.path.isfile(caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'):
    print 'CaffeNet found.'
else:
    print 'Downloading pre-trained CaffeNet model...'
    !../scripts/download_model_binary.py ../models/bvlc_reference_caffenet

CaffeNet found.


In [4]:
caffe.set_mode_cpu()

model_def = caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'
model_weights = caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'

net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)

In [5]:
# load the mean ImageNet image (as distributed with Caffe) for subtraction
mu = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
print 'mean-subtracted values:', zip('BGR', mu)

# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

mean-subtracted values: [('B', 104.0069879317889), ('G', 116.66876761696767), ('R', 122.6789143406786)]


In [6]:
# set the size of the input (we can skip this if we're happy
#  with the default; we can also change it later, e.g., for different batch sizes)
net.blobs['data'].reshape(50,        # batch size
                          3,         # 3-channel (BGR) images
                          227, 227)  # image size is 227x227

In [10]:
from PIL import Image
import glob
image_list = []
for filename in glob.glob('/home/ambika/Downloads/CrowdSourcing-Project-master/samples/*.jpg'): 
    #im=Image.open(filename)
    image_list.append(filename)

In [12]:
#image = caffe.io.load_image(caffe_root + 'examples/images/cat.jpg')
# load ImageNet labels
labels_file = caffe_root + 'data/ilsvrc12/synset_words.txt'
if not os.path.exists(labels_file):
    !../data/ilsvrc12/get_ilsvrc_aux.sh
    
labels = np.loadtxt(labels_file, str, delimiter='\t')
filename='test_labels_all.txt'
b = open(filename, 'wb') 
a = csv.writer(b)

for i in range(0,99):
    #image = caffe.io.load_image('/home/ambika/INF_project/data/dog/166995.jpg')
    print i
    image = caffe.io.load_image(image_list[i])
    transformed_image = transformer.preprocess('data', image)
    #copy the image data into the memory allocated for the net
    net.blobs['data'].data[...] = transformed_image

    ### perform classification
    output = net.forward()

    output_prob = output['prob'][0]  # the output probability vector for the first image in the batch

    print 'predicted class is:', output_prob.argmax()
    print 'output label:', labels[output_prob.argmax()]
    # sort top five predictions from softmax output
    top_inds = output_prob.argsort()[::-1][:5]  # reverse sort and take five largest items

    print 'probabilities and labels:'
    print zip(output_prob[top_inds], labels[top_inds])
    
    
    data=str(output_prob[top_inds[0]])+'\t'+labels[top_inds[0]]+'\t'+str(output_prob[top_inds[1]])+'\t'+labels[top_inds[1]]+'\t'+str(output_prob[top_inds[2]])+'\t'+labels[top_inds[2]]+'\t'+str(output_prob[top_inds[3]])+'\t'+labels[top_inds[3]]+'\t'+str(output_prob[top_inds[4]])+'\t'+labels[top_inds[4]]
    new_data=[[image_list[i]+'\t'+data]]
    a.writerows(new_data)
    #plt.imshow(image)
    
b.close()

0
predicted class is: 611
output label: n03598930 jigsaw puzzle
probabilities and labels:
[(0.50360048, 'n03598930 jigsaw puzzle'), (0.10029279, 'n02980441 castle'), (0.085219562, 'n06785654 crossword puzzle, crossword'), (0.044029515, 'n02793495 barn'), (0.034406442, 'n03877845 palace')]
1
predicted class is: 339
output label: n02389026 sorrel
probabilities and labels:
[(0.16089933, 'n02389026 sorrel'), (0.14502063, 'n02093428 American Staffordshire terrier, Staffordshire terrier, American pit bull terrier, pit bull terrier'), (0.10857349, 'n02093256 Staffordshire bullterrier, Staffordshire bull terrier'), (0.087770343, 'n02108089 boxer'), (0.0828095, 'n02109047 Great Dane')]
2
predicted class is: 15
output label: n01558993 robin, American robin, Turdus migratorius
probabilities and labels:
[(0.94511968, 'n01558993 robin, American robin, Turdus migratorius'), (0.03295581, 'n01530575 brambling, Fringilla montifringilla'), (0.0039031084, 'n01560419 bulbul'), (0.003902744, 'n01807496 par

In [23]:
print image_list[0]+'\t'+str(output_prob[top_inds[0]])+'\t'+ labels[top_inds[0]]

/home/ambika/Downloads/CrowdSourcing-Project-master/samples/26201.jpg	0.0630393	n03452741 grand piano, grand


In [22]:
print str(output_prob[top_inds[0]])+'\t'+labels[top_inds[0]]

0.0630393	n03452741 grand piano, grand
